In [ ]:
from ta.momentum import RSIIndicator
def strategy_4(data):
    def calculate_mcginley_dynamic(df, n):
        df['MD_4'] = df['close'].copy()
        for i in range(1, len(df)):
            close = df['close'].iloc[i]
            md_previous = df['MD_4'].iloc[i - 1]
            md = md_previous + (close - md_previous) / (n * (close / md_previous) ** 4)
            df.at[i, 'MD_4'] = md
        return df['MD_4']

    def rsi(df, period=16):
        indicator = RSIIndicator(close=df['close'], window=period)
        df['rsi_4'] = indicator.rsi()
        return df

    data = rsi(data)

    # Calculate ATR and ADX
    data['atr_4'] = ta.ATR(high=data['high'], low=data['low'], close=data['close'], timeperiod=16)
    data['ADX_4'] = ta.ADX(high=data['high'], low=data['low'], close=data['close'], timeperiod=14)

    data['MD_4'] = calculate_mcginley_dynamic(data, 14)
    data['macd_4'] = calculate_mcginley_dynamic(data, 12) - calculate_mcginley_dynamic(data, 26)
    data['signal_macd_4'] = data['macd_4'].ewm(span=7, adjust=False).mean()

    data['IBS_4'] = (data['close'] - data['low']) / (data['high'] - data['low'])
    data['w_4'] = data['IBS_4'].ewm(span=5, adjust=False).mean()

    def find_IRB(data):
        IRB = []
        for i in range(len(data)):
            if data['high'].iloc[i] - max(data['close'].iloc[i], data['open'].iloc[i]) < 0.55 * (data['high'].iloc[i] - data['low'].iloc[i]):
                IRB.append('IRB_UP')
            elif min(data['close'].iloc[i], data['open'].iloc[i]) - data['low'].iloc[i] > 0.55 * (data['high'].iloc[i] - data['low'].iloc[i]):
                IRB.append('IRB_DOWN')
            else:
                IRB.append('No Signal')
        data['IRB_4'] = IRB
        return data

    data = find_IRB(data)

    data['signals'] = 0

    for i in range(1, len(data)):
        if data['macd_4'].iloc[i] > data['signal_macd_4'].iloc[i] and data['ADX_4'].iloc[i] > 60:
            data['signals'].iloc[i] = 1
        elif data['macd_4'].iloc[i] < data['signal_macd_4'].iloc[i] and data['ADX_4'].iloc[i] > 60:
            data['signals'].iloc[i] = -1

    bt = Backtest(data=data)
    bt.backtest()
    x = bt.result()

    return x